# 벡터 데이터베이스

가장 손쉽게 사용할 수 있는 Chroma(크로마)와 Faiss(파이스)를 소개한다.

In [ ]:
!pip install chromadb faiss-gpu

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

## Chroma(크로마)

In [2]:
import urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings -> will be deprecated
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

#### 준비 사항

- PDF를 로드하는 -> PyPDFLoader
- 문서들을 다수의 청크로 분할하는 -> RecursiveCharacterTextSplitter
- 청크들을 임베딩 벡터로 변환 시 OpenAI 의 Embedding API를 사용하기 위한 -> OpenAIEmbeddings
- 임베딩 벡터를 적재하기 위한 벡터 데이터베이스 -> Chroma 또는 Faiss

In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

In [3]:
loader = PyPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 445


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [5]:
splited_docs = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(splited_docs))

분할된 청크의 수: 497


In [6]:
chunks = [splited_doc.page_content for splited_doc in splited_docs]
print('청크의 최대 길이 :',max(len(chunk) for chunk in chunks))

청크의 최대 길이 : 1000


In [26]:
del db
del db_to_file

In [7]:
db = Chroma.from_documents(splited_docs, OpenAIEmbeddings())
print('문서의 수:', db._collection.count())

문서의 수: 497


In [8]:
question = '북한의 교육과정'
docs = db.similarity_search(question)
print('문서의 수:', len(docs))

문서의 수: 4


In [9]:
for doc in docs:
  print(doc)
  print('--' * 10)

page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'page': 41, 'source': '2023_북한인권보고서.pdf'}
--------------------
page_content='309		북한의	학제는	2012년	전반적	의무교육(유치원

In [17]:
import shutil
shutil.rmtree('./chroma_db', ignore_errors=True)

In [18]:
# 파일로 저장
db_to_file = Chroma.from_documents(splited_docs, OpenAIEmbeddings(), persist_directory = './chroma_db')
print('문서의 수:', db_to_file._collection.count())

문서의 수: 497


In [21]:
!pip install -U langchain langchain-chroma

In [22]:
db_from_file = Chroma(persist_directory='./chroma_db',
		      embedding_function=OpenAIEmbeddings())

In [23]:
print('문서의 수:', db_from_file._collection.count())

문서의 수: 497


### 검색해보자

In [25]:
question = '북한의 교육 과정'
docs = db_from_file.similarity_search_with_relevance_scores(question, k=3)

for doc in docs:
  print(doc)
  print('--' * 10)

(Document(metadata={'page': 41, 'source': '2023_북한인권보고서.pdf'}, page_content='2023 북한인권보고서\n40\n명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 \n없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절\n히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모\n의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 \n제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또\n한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실\n탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 \n하고 있다.\n북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 \n연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제\n도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로'), 0.828519056218757)
---------

## Faiss

In [26]:
faiss_db = FAISS.from_documents(splited_docs, OpenAIEmbeddings())
print('문서의 수:', faiss_db.index.ntotal)

문서의 수: 497


In [27]:
faiss_db.save_local('./faiss_db')

new_db_faiss = FAISS.load_local('./faiss_db',
				OpenAIEmbeddings(),
				allow_dangerous_deserialization=True)

### 검색해보자 - Faiss

In [28]:
question = '북한의 교육 과정'
docs = new_db_faiss.similarity_search(question, k=3)

for doc in docs:
  print(doc)
  print('--' * 10)

page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'source': '2023_북한인권보고서.pdf', 'page': 41}
--------------------
page_content='309		북한의	학제는	2012년	전반적	의무교육(유치원